In [1]:
import math
import pandas as pd
import numpy as np
import scipy
import random
import math 
from scipy import stats
from csv import reader
random.seed(120)

In [2]:
# path1 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Bonds/Data/historical parameters.csv"
# path2 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Bonds/Data/rates_instrument.csv"
# path3 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Bonds/Data/rates_exchange.csv"
# path4 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Bonds/Data/rates_interest.csv"
# path5 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Bonds/Data/tenor.csv"
# path6 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Bonds/Data/transaction_bond_hist_d.csv"
# path7 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Bonds/Data/spread_rate.csv"


In [3]:
path1 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/historical parameters.csv"
path2 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_instrument.csv"
path3 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_exchange.csv"
path4 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_interest.csv"
path5 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/tenor.csv"
path6 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/transaction_bond_hist_d.csv"
path7 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/spread_rate.csv"
path8 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_interest_hp.csv"

In [4]:
def getData(path, fileext = None):
    if fileext == ".csv":
        df = pd.read_csv(path)
    elif fileext == ".xlsx" or fileext == ".xls":
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path)
    return df

In [5]:
hp = getData(path1)
n = hp['no_of_days']
n = list(n)
n = n[0]
decay_factor = hp['decay_factor']
holding_period = hp['holding_period']
confidence_level = hp['confidence_level']
confidence_level = list(confidence_level)
confidence_level = confidence_level[0]
decay_factor = list(decay_factor)
decay_factor = decay_factor[0]

In [6]:
hp

,decay_factor,holding_period,confidence_level,no_of_days,is_scenario_stressed
0,0,1,0.99,250,0


# Rates Instrument

In [7]:
##Importing dataset rates_instrument
rates_instrument= getData(path2)

In [8]:
#Function to get a specific type of data from the particular column in data(Filtering)
def loadData(df, colname, types = None):
    
    res_df = df[df[colname] == types].reset_index(drop=True)
    return res_df
    

In [9]:
rates_instrument_D = loadData(rates_instrument,'instrument_type',types = 'D')
rates_instrument_E = loadData(rates_instrument,'instrument_type',types = 'E')

# Rates Exchange

In [10]:
rates_exchange = getData(path3) 

In [11]:
rates_exchange

,date,currency_from,currency_to,rate,rate_stressed
0,2019-12-31,INR,INR,1,1
1,2019-12-30,INR,INR,1,1
2,2019-12-27,INR,INR,1,1
3,2019-12-26,INR,INR,1,1
4,2019-12-24,INR,INR,1,1
...,...,...,...,...,...
246,2018-12-26,INR,INR,1,1
247,2018-12-24,INR,INR,1,1
248,2018-12-21,INR,INR,1,1
249,2018-12-20,INR,INR,1,1


In [12]:
#Generating key
rates_exchange["key"] = rates_exchange["currency_from"] + '_' + rates_exchange["currency_to"]
rates_exchange

,date,currency_from,currency_to,rate,rate_stressed,key
0,2019-12-31,INR,INR,1,1,INR_INR
1,2019-12-30,INR,INR,1,1,INR_INR
2,2019-12-27,INR,INR,1,1,INR_INR
3,2019-12-26,INR,INR,1,1,INR_INR
4,2019-12-24,INR,INR,1,1,INR_INR
...,...,...,...,...,...,...
246,2018-12-26,INR,INR,1,1,INR_INR
247,2018-12-24,INR,INR,1,1,INR_INR
248,2018-12-21,INR,INR,1,1,INR_INR
249,2018-12-20,INR,INR,1,1,INR_INR


# Rate Interest

In [13]:
rates_interest = getData(path4)

In [14]:
#Generating key between curve and tenor
rates_interest["key"] = rates_interest["curve"] + '_' + rates_interest["tenor"]
rates_interest.head()

,date,curve,tenor,rate,rate_stressed,key
0,2019-12-31,INRZeroCurve,1m,4.95,4.95,INRZeroCurve_1m
1,2019-12-30,INRZeroCurve,1m,4.90,4.90,INRZeroCurve_1m
2,2019-12-27,INRZeroCurve,1m,4.90,4.90,INRZeroCurve_1m
3,2019-12-26,INRZeroCurve,1m,4.90,4.90,INRZeroCurve_1m
4,2019-12-24,INRZeroCurve,1m,4.90,4.90,INRZeroCurve_1m


In [15]:
##Adding current rate to the top of the rates
def sim_returns(df):
    simple_returns = pd.DataFrame([])

    for a, g in df.groupby('key'):
        
        g = g.reset_index(drop = True)
        g['simple_returns'] = g['rate']-g['rate'].shift(-1)
        #new_row = g.iloc[0].to_dict()
        #new_row['simple_returns'] = [0]
        #top_row = pd.DataFrame(new_row)
        g['simulated rates'] =( g['simple_returns'] + g['rate'].iloc[0]).shift(fill_value = g['rate'][0])
        #g = g.iloc[:-1 , :]
        #g = pd.concat([top_row, g]).reset_index(drop = True)

        simple_returns = simple_returns.append(g)
        
    
    return simple_returns

In [16]:
simulated_interest = sim_returns(rates_interest)
simulated_interest.head()

/tmp/ipykernel_316/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_316/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_316/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_316/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_316/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

,date,curve,tenor,rate,rate_stressed,key,simple_returns,simulated rates
0,2019-12-31,INRZeroCurve,025m,5.04,5.04,INRZeroCurve_025m,0.04,5.04
1,2019-12-30,INRZeroCurve,025m,5.00,5.00,INRZeroCurve_025m,0.04,5.08
2,2019-12-27,INRZeroCurve,025m,4.96,4.96,INRZeroCurve_025m,-0.05,5.08
3,2019-12-26,INRZeroCurve,025m,5.01,5.01,INRZeroCurve_025m,-0.01,4.99
4,2019-12-24,INRZeroCurve,025m,5.02,5.02,INRZeroCurve_025m,0.03,5.03


In [17]:
simulated_interest

,date,curve,tenor,rate,rate_stressed,key,simple_returns,simulated rates
0,2019-12-31,INRZeroCurve,025m,5.040,5.040,INRZeroCurve_025m,0.040,5.040
1,2019-12-30,INRZeroCurve,025m,5.000,5.000,INRZeroCurve_025m,0.040,5.080
2,2019-12-27,INRZeroCurve,025m,4.960,4.960,INRZeroCurve_025m,-0.050,5.080
3,2019-12-26,INRZeroCurve,025m,5.010,5.010,INRZeroCurve_025m,-0.010,4.990
4,2019-12-24,INRZeroCurve,025m,5.020,5.020,INRZeroCurve_025m,0.030,5.030
...,...,...,...,...,...,...,...,...
246,2018-12-26,INRZeroCurve,9yr,7.371,7.371,INRZeroCurve_9yr,-0.002,6.760
247,2018-12-24,INRZeroCurve,9yr,7.373,7.373,INRZeroCurve_9yr,0.035,6.782
248,2018-12-21,INRZeroCurve,9yr,7.338,7.338,INRZeroCurve_9yr,0.023,6.819
249,2018-12-20,INRZeroCurve,9yr,7.315,7.315,INRZeroCurve_9yr,0.000,6.807


In [18]:
simulated_interest.iloc[250]

date                      2018-12-19
curve                   INRZeroCurve
tenor                           025m
rate                             6.6
rate_stressed                    6.6
key                INRZeroCurve_025m
simple_returns                   NaN
simulated rates                 5.03
Name: 250, dtype: object

# Tenors

In [19]:
tenors = getData(path5)


# Transaction file

In [20]:
tx_bond = getData(path6)
tx_bond

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_tenor,lower_tenor_val,upper_tenor_val,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,050m,025m,050m,91,182,INRZeroCurve,1,1,0,1
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,100yr,075m,100yr,274,366,INRZeroCurve,1,1,0,1
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,150yr,125yr,150yr,456,547,INRZeroCurve,1,1,0,1
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,200yr,175yr,200yr,639,731,INRZeroCurve,1,1,0,1
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,250yr,225yr,250yr,821,912,INRZeroCurve,1,1,0,1
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,250yr,225yr,250yr,821,912,INRZeroCurve,1,1,0,1
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,050m,025m,050m,91,182,INRZeroCurve,1,1,0,2
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,125yr,100yr,125yr,366,456,INRZeroCurve,1,1,0,2
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,225yr,200yr,225yr,731,821,INRZeroCurve,1,1,0,2
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,325yr,300yr,325yr,1096,1186,INRZeroCurve,1,1,0,2


In [21]:
#print(tx_bond["instrument"],_ ,tx_bond["k"])
tx_bond["key"] = tx_bond['instrument'].map(str) + '_' + tx_bond['k'].map(str) 
tx_bond

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,lower_tenor_val,upper_tenor_val,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,025m,050m,91,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,075m,100yr,274,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,125yr,150yr,456,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,175yr,200yr,639,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,225yr,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,225yr,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,025m,050m,91,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,100yr,125yr,366,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,200yr,225yr,731,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,300yr,325yr,1096,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4


In [22]:
##Sorting data by'key' column
tx_bond= tx_bond.sort_values(by=['key']).reset_index(drop=True)
tx_bond

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,lower_tenor_val,upper_tenor_val,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,025m,050m,91,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,075m,100yr,274,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,125yr,150yr,456,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,175yr,200yr,639,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,225yr,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,225yr,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,025m,050m,91,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,100yr,125yr,366,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,200yr,225yr,731,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,300yr,325yr,1096,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4


In [23]:
#Converting to correct datetime format
tx_bond["c_date"] = pd.to_datetime(tx_bond["current_date"],format =  "%d-%m-%Y")
tx_bond.head(10)

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_tenor_val,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,050m,91,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1,2020-04-20
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,100yr,274,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2,2020-04-20
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,150yr,456,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3,2020-04-20
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,200yr,639,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4,2020-04-20
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5,2020-04-20
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6,2020-04-20
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,050m,91,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1,2020-04-20
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,125yr,366,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2,2020-04-20
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,225yr,731,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3,2020-04-20
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,325yr,1096,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4,2020-04-20


In [24]:
#Adding c_date and residual_maturity_expected column
tx_bond['m_date_E'] = tx_bond['c_date'] + pd.to_timedelta(tx_bond['residual_maturity_expected'], unit='d')
tx_bond.head(10)

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,91,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,274,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,456,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,639,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,91,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,366,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,731,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3,2020-04-20,2022-08-15
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,1096,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4,2020-04-20,2023-08-15


In [25]:
#Adding c_date and residual_maturity_Actual column
tx_bond['m_date_A'] = tx_bond['c_date'] + pd.to_timedelta(tx_bond['residual_maturity_Actual'], unit='d')
tx_bond

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3,2020-04-20,2022-08-15,2022-08-15
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4,2020-04-20,2023-08-15,2023-08-15


In [26]:
tx_bond["lower_tenor_key"] = tx_bond["curve"]+ "_" + tx_bond["lower_tenor"]
tx_bond.head()

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr


In [27]:
tx_bond["upper_tenor_key"] = tx_bond["curve"]+ "_" + tx_bond["upper_tenor"]
tx_bond

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,1,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,1,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,1,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,1,1,0,2,IN1520190101NCD_3,2020-04-20,2022-08-15,2022-08-15,INRZeroCurve_200yr,INRZeroCurve_225yr
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,1,1,0,2,IN1520190101NCD_4,2020-04-20,2023-08-15,2023-08-15,INRZeroCurve_300yr,INRZeroCurve_325yr


In [28]:
lower_tenor_key_list = list(tx_bond["lower_tenor_key"])
upper_tenor_key_list = list(tx_bond["upper_tenor_key"])
tenor_list = lower_tenor_key_list + upper_tenor_key_list
tenor_list = set(tenor_list)
tenor_list = list(tenor_list)
tenor_df = pd.DataFrame(tenor_list).reset_index(drop = True)
tenor_df = tenor_df.rename(columns = {0:'tenor_list'})

In [29]:
tenor_df

,tenor_list
0,INRZeroCurve_075m
1,INRZeroCurve_100yr
2,INRZeroCurve_125yr
3,INRZeroCurve_1m
4,INRZeroCurve_325yr
5,INRZeroCurve_250yr
6,INRZeroCurve_200yr
7,INRZeroCurve_050m
8,INRZeroCurve_300yr
9,INRZeroCurve_150yr


In [30]:
#tenor_list_unique = tenor_list_unique.rename(columns = {0:'tenor_list_unique'})
tenor_df = tenor_df.sort_values('tenor_list')
tenor_df = tenor_df.reset_index(drop=True)

Matching the key values of simulated_interest dataframe with the tenor_list_unique key values of tenor_list_unique dataframe 
and dropped out the columns 'key','date','rate_stressed','simple_returns','simulated rates'.

In [31]:
rates_interest_current = pd.merge(tenor_df,simulated_interest,how='left',left_on='tenor_list',right_on='key').drop(['key','date','rate_stressed','simple_returns','rate'], axis=1)

In [32]:
rates_interest_current.head(50)

,tenor_list,curve,tenor,simulated rates
0,INRZeroCurve_025m,INRZeroCurve,025m,5.04
1,INRZeroCurve_025m,INRZeroCurve,025m,5.08
2,INRZeroCurve_025m,INRZeroCurve,025m,5.08
3,INRZeroCurve_025m,INRZeroCurve,025m,4.99
4,INRZeroCurve_025m,INRZeroCurve,025m,5.03
5,INRZeroCurve_025m,INRZeroCurve,025m,5.07
6,INRZeroCurve_025m,INRZeroCurve,025m,5.02
7,INRZeroCurve_025m,INRZeroCurve,025m,5.06
8,INRZeroCurve_025m,INRZeroCurve,025m,5.03
9,INRZeroCurve_025m,INRZeroCurve,025m,5.01


In [33]:
from copy import deepcopy

In [34]:
test_rate_df = deepcopy(rates_interest_current)
test_rate_df


,tenor_list,curve,tenor,simulated rates
0,INRZeroCurve_025m,INRZeroCurve,025m,5.040
1,INRZeroCurve_025m,INRZeroCurve,025m,5.080
2,INRZeroCurve_025m,INRZeroCurve,025m,5.080
3,INRZeroCurve_025m,INRZeroCurve,025m,4.990
4,INRZeroCurve_025m,INRZeroCurve,025m,5.030
...,...,...,...,...
3258,INRZeroCurve_325yr,INRZeroCurve,325yr,6.367
3259,INRZeroCurve_325yr,INRZeroCurve,325yr,6.307
3260,INRZeroCurve_325yr,INRZeroCurve,325yr,6.367
3261,INRZeroCurve_325yr,INRZeroCurve,325yr,6.357


In [35]:
rates_interest_current_1V = (test_rate_df.drop_duplicates(['tenor_list']))
rates_interest_current_1V = rates_interest_current_1V.reset_index(drop=True)
rates_interest_current_1V

,tenor_list,curve,tenor,simulated rates
0,INRZeroCurve_025m,INRZeroCurve,025m,5.040
1,INRZeroCurve_050m,INRZeroCurve,050m,5.170
2,INRZeroCurve_075m,INRZeroCurve,075m,5.190
3,INRZeroCurve_100yr,INRZeroCurve,100yr,5.190
4,INRZeroCurve_125yr,INRZeroCurve,125yr,5.190
5,INRZeroCurve_150yr,INRZeroCurve,150yr,6.110
6,INRZeroCurve_175yr,INRZeroCurve,175yr,5.190
7,INRZeroCurve_1m,INRZeroCurve,1m,4.950
8,INRZeroCurve_200yr,INRZeroCurve,200yr,5.776
9,INRZeroCurve_225yr,INRZeroCurve,225yr,5.776


In [36]:
###Filtering according to Instrument
rates_interest_current_1S = rates_interest_current.filter(['tenor_list', 'simulated rates']) ##To extract particular columns
rates_interest_current_1S

,tenor_list,simulated rates
0,INRZeroCurve_025m,5.040
1,INRZeroCurve_025m,5.080
2,INRZeroCurve_025m,5.080
3,INRZeroCurve_025m,4.990
4,INRZeroCurve_025m,5.030
...,...,...
3258,INRZeroCurve_325yr,6.367
3259,INRZeroCurve_325yr,6.307
3260,INRZeroCurve_325yr,6.367
3261,INRZeroCurve_325yr,6.357


In [37]:
rates_interest_current_1S = rates_interest_current_1S.set_index('tenor_list')
rates_interest_current_1S

,simulated rates
tenor_list,
INRZeroCurve_025m,5.040
INRZeroCurve_025m,5.080
INRZeroCurve_025m,5.080
INRZeroCurve_025m,4.990
INRZeroCurve_025m,5.030
...,...
INRZeroCurve_325yr,6.367
INRZeroCurve_325yr,6.307
INRZeroCurve_325yr,6.367


In [38]:
##Fetching current values for each of the tenor_list_unique

In [39]:
##Segregating simulations according to the tenor_list_unique(unique key) generated
df = (rates_interest_current_1S.assign(labels = rates_interest_current_1S.groupby(level = 0).cumcount())
            .groupby([rates_interest_current_1S.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))
df

,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,...,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates
tenor_list,,,,,,,,,,,,,,,,,,,,,
INRZeroCurve_025m,5.040,5.080,5.080,4.990,5.030,5.070,5.020,5.060,5.030,5.010,...,5.020,5.050,4.900,5.060,5.110,5.040,4.960,5.040,5.190,5.030
INRZeroCurve_050m,5.170,5.140,5.200,5.180,5.130,5.180,5.150,5.190,5.180,5.150,...,5.150,5.120,5.160,5.180,5.160,5.190,5.190,5.150,5.150,5.160
INRZeroCurve_075m,5.190,5.160,5.150,5.200,5.150,5.220,5.130,5.200,5.200,5.230,...,5.190,5.190,5.190,5.190,5.190,5.190,5.190,5.190,5.190,5.190
INRZeroCurve_100yr,5.190,5.160,5.150,5.200,5.150,5.220,5.130,5.200,5.200,5.230,...,5.170,5.160,5.200,5.180,5.200,5.200,5.140,5.200,5.190,5.190
INRZeroCurve_125yr,5.190,5.160,5.150,5.200,5.150,5.220,5.130,5.200,5.200,5.230,...,5.170,5.160,5.200,5.180,5.200,5.200,5.140,5.200,5.190,5.190
INRZeroCurve_150yr,6.110,6.110,6.110,6.110,6.110,6.110,6.110,6.110,6.110,6.110,...,6.090,6.080,6.120,6.100,6.120,6.120,6.060,6.120,6.110,6.110
INRZeroCurve_175yr,5.190,5.160,5.150,5.200,5.150,5.220,5.130,5.200,5.200,5.230,...,5.170,5.160,5.200,5.180,5.200,5.200,5.140,5.200,5.190,5.190
INRZeroCurve_1m,4.950,5.000,4.950,4.950,4.950,4.950,4.880,4.970,4.970,4.910,...,4.930,4.960,4.840,4.910,4.950,4.960,5.000,4.890,5.020,4.940
INRZeroCurve_200yr,5.776,5.721,5.787,5.808,5.755,5.748,5.735,5.835,5.809,5.741,...,5.729,5.798,5.743,5.788,5.779,5.743,5.824,5.723,5.793,5.776


In [40]:
#Extracting values at 0th index
tenor_rates = df.iloc[:,0]
tenor_rates

tenor_list
INRZeroCurve_025m     5.040
INRZeroCurve_050m     5.170
INRZeroCurve_075m     5.190
INRZeroCurve_100yr    5.190
INRZeroCurve_125yr    5.190
INRZeroCurve_150yr    6.110
INRZeroCurve_175yr    5.190
INRZeroCurve_1m       4.950
INRZeroCurve_200yr    5.776
INRZeroCurve_225yr    5.776
INRZeroCurve_250yr    5.776
INRZeroCurve_300yr    6.357
INRZeroCurve_325yr    6.357
Name: simulated rates, dtype: float64

In [41]:
tenor_rates_df_V = pd.DataFrame(tenor_rates)

In [42]:
tenor_rates_df_V

,simulated rates
tenor_list,
INRZeroCurve_025m,5.040
INRZeroCurve_050m,5.170
INRZeroCurve_075m,5.190
INRZeroCurve_100yr,5.190
INRZeroCurve_125yr,5.190
INRZeroCurve_150yr,6.110
INRZeroCurve_175yr,5.190
INRZeroCurve_1m,4.950
INRZeroCurve_200yr,5.776


In [43]:
tenor_rates_df_V.index.rename('key',inplace=True)

In [44]:
tenor_rates_df_V

,simulated rates
key,
INRZeroCurve_025m,5.040
INRZeroCurve_050m,5.170
INRZeroCurve_075m,5.190
INRZeroCurve_100yr,5.190
INRZeroCurve_125yr,5.190
INRZeroCurve_150yr,6.110
INRZeroCurve_175yr,5.190
INRZeroCurve_1m,4.950
INRZeroCurve_200yr,5.776


Match lower tenor key with the tenor_rate df and get the corresponding rate values.
Match upper tenor key with the tenor_rate df and get the corresponding rate values


In [45]:
tx_bond_new = tx_bond.merge(tenor_rates_df_V,how='inner',left_on='lower_tenor_key',right_on='key')

In [46]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,simulated rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,1,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,1,0,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190


In [47]:
##Renaming the rate column
tx_bond_new = tx_bond_new.rename(columns = {'simulated rates':'lower_rates'})
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,1,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,1,0,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190


In [48]:
#Upper rates
tx_bond_new = tx_bond_new.merge(tenor_rates_df_V, how='inner', left_on='upper_tenor_key', right_on='key')

In [49]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,simulated rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190


In [50]:

tx_bond_new =  tx_bond_new.rename(columns = {'simulated rates':'upper_rates'})

In [51]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190


In [52]:
#Sorting values by lower-rates
tx_bond_new = tx_bond_new.sort_values(by=['lower_rates'])


In [53]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates
15,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_1,2020-04-20,2020-06-19,2020-06-21,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
16,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_3,2020-04-20,2020-12-19,2020-12-20,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190
10,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_5,2020-04-20,2021-06-19,2021-06-20,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190


In [54]:
# Interpolated rates

In [55]:
def interpolate_values(df):

    numerator1 = (df['upper_rates'] - df['lower_rates'])
    numerator2 = (df['residual_maturity_expected'] - df['lower_maturity'])
    numerator = numerator1*numerator2
    denominator = (df['upper_maturity'] - df['lower_maturity'])
    interpolate_col = df['lower_rates'] + (numerator/denominator)
    return interpolate_col


In [56]:
tx_bond_new['Interpolated_rates'] = tx_bond_new.apply(lambda x: x['upper_rates'] if x['upper_tenor_key'] == x['lower_tenor_key'] else interpolate_values(x), axis=1)

In [57]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates
15,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_1,2020-04-20,2020-06-19,2020-06-21,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040,4.993500
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143
16,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_3,2020-04-20,2020-12-19,2020-12-20,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190,5.183261
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000
10,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_5,2020-04-20,2021-06-19,2021-06-20,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000


### SPREAD's

In [58]:
spreads= getData(path7)
spreads

,date,instrument,rate,rate_stressed
0,2019-12-31,IN1520190101NCD,5.443980,5.443980
1,2019-12-30,IN1520190101NCD,5.273412,5.273412
2,2019-12-27,IN1520190101NCD,5.299568,5.299568
3,2019-12-26,IN1520190101NCD,5.290016,5.290016
4,2019-12-24,IN1520190101NCD,5.510400,5.510400
...,...,...,...,...
1501,2018-12-26,INE134E08925PSUT,0.949293,0.949293
1502,2018-12-24,INE134E08925PSUT,0.949293,0.949293
1503,2018-12-21,INE134E08925PSUT,0.949293,0.949293
1504,2018-12-20,INE134E08925PSUT,0.949293,0.949293


In [59]:
##Dropping duplicates
spread_rates = (spreads.drop_duplicates(['instrument']))
spread_rates.columns= spread_rates.columns.str.lower() ##Column names to lower case
spread_rates.head()

,date,instrument,rate,rate_stressed
0,2019-12-31,IN1520190101NCD,5.443980,5.443980
251,2019-12-31,IN1520190101PF,1.123651,1.123651
502,2019-12-31,IN1520190101RPF,1.041761,1.041761
753,2019-12-31,INE036D08015SD,3.329692,3.329692
1004,2019-12-31,INE095H07014PSUTF,4.649538,4.649538


In [60]:
##Renaming rates to spread_rates
spread_rates =  spread_rates.rename(columns = {'rate':'spread_rates'})
spread_rates.head()

,date,instrument,spread_rates,rate_stressed
0,2019-12-31,IN1520190101NCD,5.443980,5.443980
251,2019-12-31,IN1520190101PF,1.123651,1.123651
502,2019-12-31,IN1520190101RPF,1.041761,1.041761
753,2019-12-31,INE036D08015SD,3.329692,3.329692
1004,2019-12-31,INE095H07014PSUTF,4.649538,4.649538


In [61]:
s_rate_df = spread_rates[['instrument', 'spread_rates']].reset_index(drop=True)

In [62]:
s_rate_df

,instrument,spread_rates
0,IN1520190101NCD,5.443980
1,IN1520190101PF,1.123651
2,IN1520190101RPF,1.041761
3,INE036D08015SD,3.329692
4,INE095H07014PSUTF,4.649538
5,INE134E08925PSUT,0.602718


In [63]:
tx_bond_revamp = tx_bond_new.merge(s_rate_df, on ='instrument', how='left')

In [64]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates
0,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,IN1520190101PF_1,2020-04-20,2020-06-19,2020-06-21,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040,4.993500,1.123651
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,NaN
2,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980
3,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143,1.123651
4,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,IN1520190101PF_3,2020-04-20,2020-12-19,2020-12-20,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190,5.183261,1.123651
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,NaN
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,NaN
7,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,NaN
8,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980
9,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,IN1520190101PF_5,2020-04-20,2021-06-19,2021-06-20,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,1.123651


In [65]:
#Replacing NAN's with 0's
tx_bond_revamp['spread_rates'] = tx_bond_revamp['spread_rates'].fillna(0)
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates
0,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,IN1520190101PF_1,2020-04-20,2020-06-19,2020-06-21,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040,4.993500,1.123651
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000
2,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980
3,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143,1.123651
4,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,IN1520190101PF_3,2020-04-20,2020-12-19,2020-12-20,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190,5.183261,1.123651
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000
7,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000
8,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980
9,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,IN1520190101PF_5,2020-04-20,2021-06-19,2021-06-20,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,1.123651


In [66]:
#YTM(Yield to Maturity)
tx_bond_revamp['YTM']= tx_bond_revamp['Interpolated_rates'] + tx_bond_revamp['spread_rates'] + tx_bond_revamp['mark_up_value'] 

In [67]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM
0,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,2020-04-20,2020-06-19,2020-06-21,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040,4.993500,1.123651,6.117151
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000
2,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123
3,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143,1.123651,6.250794
4,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,2020-04-20,2020-12-19,2020-12-20,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190,5.183261,1.123651,6.306912
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543
7,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000
8,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980
9,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,2020-04-20,2021-06-19,2021-06-20,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,1.123651,6.313651


In [68]:
#result = tx_bond_revamp.dtypes
#result

In [69]:
tx_bond_revamp['maturity_period_E'] = (tx_bond_revamp['m_date_E'] - tx_bond_revamp['c_date']).dt.days
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E
0,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,2020-06-19,2020-06-21,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040,4.993500,1.123651,6.117151,60
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105
2,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117
3,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143,1.123651,6.250794,152
4,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,2020-12-19,2020-12-20,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190,5.183261,1.123651,6.306912,243
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654
7,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289
8,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482
9,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,2021-06-19,2021-06-20,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,1.123651,6.313651,425


In [70]:
tx_bond_revamp['maturity_period_A'] = (tx_bond_revamp['m_date_A'] - tx_bond_revamp['c_date']).dt.days
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A
0,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,2020-06-21,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040,4.993500,1.123651,6.117151,60,62
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105,105
2,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117,118
3,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143,1.123651,6.250794,152,153
4,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,2020-12-20,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190,5.183261,1.123651,6.306912,243,244
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470,470
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654,654
7,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289,289
8,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482,482
9,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,2021-06-20,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,1.123651,6.313651,425,426


In [71]:
##Condition

In [72]:
tx_bond_revamp['no_of_days_in_a_year'] = ''
for i in range (len(tx_bond_revamp)):

  
  if  tx_bond_revamp['intbasis'][i] == 0:
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360
  
  elif tx_bond_revamp['intbasis'][i] == 1: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 365  
  
  elif tx_bond_revamp['intbasis'][i] == 2: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360 
  
  elif tx_bond_revamp['intbasis'][i] == 3: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 365 
  
  elif tx_bond_revamp['intbasis'][i] == 4: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360
   
  else:
        pass
  

/tmp/ipykernel_316/2565406497.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tx_bond_revamp['no_of_days_in_a_year'][i] = 365
/tmp/ipykernel_316/2565406497.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tx_bond_revamp['no_of_days_in_a_year'][i] = 360


In [73]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year
0,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040,4.993500,1.123651,6.117151,60,62,365
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105,105,365
2,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117,118,360
3,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143,1.123651,6.250794,152,153,365
4,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190,5.183261,1.123651,6.306912,243,244,365
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470,470,365
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654,654,365
7,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289,289,365
8,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482,482,360
9,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,1.123651,6.313651,425,426,365


In [74]:
#cashflow_mtm
tx_bond_revamp['cashflow_mtm'] = tx_bond_revamp['Cashflow'] / ((1+(tx_bond_revamp['YTM']/100))**(tx_bond_revamp['residual_maturity_Actual']/tx_bond_revamp['no_of_days_in_a_year'])) 

In [75]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm
0,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,INRZeroCurve_025m,4.950,5.040,4.993500,1.123651,6.117151,60,62,365,4949.826628
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105,105,365,3943.601917
2,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117,118,360,12338.710772
3,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,INRZeroCurve_050m,5.040,5.170,5.127143,1.123651,6.250794,152,153,365,4874.523069
4,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,INRZeroCurve_075m,5.170,5.190,5.183261,1.123651,6.306912,243,244,365,4799.697001
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470,470,365,3741.210574
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654,654,365,3647.372389
7,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289,289,365,3842.917306
8,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482,482,360,11136.489436
9,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,INRZeroCurve_125yr,5.190,5.190,5.190000,1.123651,6.313651,425,426,365,4655.189072


In [76]:
##Sorting data by'key' column
tx_bond_revamp= tx_bond_revamp.sort_values(by=['key']).reset_index(drop=True)
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105,105,365,3943.601917
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289,289,365,3842.917306
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470,470,365,3741.210574
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654,654,365,3647.372389
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835,365,3517.792617
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835,365,87944.81543
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117,118,360,12338.710772
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482,482,360,11136.489436
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980,11.219980,847,847,360,9927.783236
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980,11.800980,1212,1212,360,8758.101185


In [77]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105,105,365,3943.601917
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289,289,365,3842.917306
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470,470,365,3741.210574
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654,654,365,3647.372389
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835,365,3517.792617
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835,365,87944.81543
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117,118,360,12338.710772
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482,482,360,11136.489436
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980,11.219980,847,847,360,9927.783236
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980,11.800980,1212,1212,360,8758.101185


In [78]:
##MTM

In [79]:
df2 = tx_bond_revamp.groupby('id')['cashflow_mtm'].sum()
mtm = list(df2)
mtm = [round(item, 0) for item in mtm]
mtm = pd.DataFrame(mtm)
mtm

,0
0,106638.0
1,145198.0
2,210214.0


In [80]:
def get_mtm_bc(tx_eq):
    
    res_df = list()
    for k, h in tx_eq.groupby('id'):
        result = {'idx':h["id"].iloc[0] , "env":k,"instrument":h["instrument"].iloc[0] ,"currency":h["currency"].iloc[0] ,"currency_base":h["currency_base"].iloc[0], "face_value":h["Face_value"].iloc[0], "maturity_amount":h["Maturity_Amount"].iloc[0]}
        res_df.append(result)
        #res_df = pd.DataFrame(res_df)

    return res_df

In [81]:
#, "bondwise_current_mtm":mtm[0]
red = get_mtm_bc(tx_bond_revamp)
res = pd.DataFrame(red)
res

,idx,env,instrument,currency,currency_base,face_value,maturity_amount
0,CGS001,CGS001,IN0020100015CGS,INR,INR,100,100000
1,NCD001,NCD001,IN1520190101NCD,INR,INR,100,150000
2,PF001,PF001,IN1520190101PF,INR,INR,100,200000


In [82]:
#renaming the column for mtm
mtm.rename(columns = {0:'bondwise_current_mtm'}, inplace = True)
mtm = mtm.reset_index(drop = True)
mtm

,bondwise_current_mtm
0,106638.0
1,145198.0
2,210214.0


In [83]:
#concatenating res dataframe and mtm dataframe 
result_new = pd.concat([res, mtm], axis = 1)
result_new

,idx,env,instrument,currency,currency_base,face_value,maturity_amount,bondwise_current_mtm
0,CGS001,CGS001,IN0020100015CGS,INR,INR,100,100000,106638.0
1,NCD001,NCD001,IN1520190101NCD,INR,INR,100,150000,145198.0
2,PF001,PF001,IN1520190101PF,INR,INR,100,200000,210214.0


In [84]:
#Calibrating price
result_new['price'] = (result_new['bondwise_current_mtm']*result_new['face_value'])/result_new['maturity_amount']
result_new

,idx,env,instrument,currency,currency_base,face_value,maturity_amount,bondwise_current_mtm,price
0,CGS001,CGS001,IN0020100015CGS,INR,INR,100,100000,106638.0,106.638000
1,NCD001,NCD001,IN1520190101NCD,INR,INR,100,150000,145198.0,96.798667
2,PF001,PF001,IN1520190101PF,INR,INR,100,200000,210214.0,105.107000


### All MTMS

In [85]:
interpolated_curve_rate = np.empty((n+1,1))

In [86]:
lW_rate1 = np.empty((n+1,3))
up_rate1 = np.empty((n+1,3))

In [87]:
interpolated_curve_rate = pd.DataFrame()
lw_rate1 = pd.DataFrame()
up_rate1 = pd.DataFrame()
for i in range(0,len(tx_bond_revamp)):
    
    a = tx_bond_revamp.loc[i,['lower_tenor_key', 'upper_tenor_key']]
    a_a = list(a)
    rates = simulated_interest[simulated_interest['key'].isin(a)]
    rates = rates.loc[:, ['key','simulated rates']]
    rates = rates.set_index('key')
        ##Segregating simulations according to the instrument
    rates2 = (rates.assign(labels = rates.groupby(level = 0).cumcount())
            .groupby([rates.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))

    rates2 = rates2.transpose()    
    rates2 = rates2.reset_index(drop = True)
    rates2 = rates2.loc[:, [a_a[0],a_a[1]]]
    #print(rates2)
    #rates2 =rates2[,match(a,colnames(rates2))]
    rates2['key'] = tx_bond_revamp['key'][i]
    rates2['maturity']= tx_bond_revamp['residual_maturity_expected'][i]
    rates2['lower_tenor'] = tx_bond_revamp['lower_maturity'][i]
    rates2['upper_tenor'] = tx_bond_revamp['upper_maturity'][i]
    if (rates2.iat[0,4]==rates2.iat[0,5]):
        rates2['intepolated'] = rates2.iloc[:,0]

    else:
        rates2['intepolated'] = rates2.iloc[:,0]+((rates2.iloc[:,1]-rates2.iloc[:,0])*(rates2.iloc[:,3]-rates2.iloc[:,4]))/(rates2.iloc[:,5]-rates2.iloc[:,4])

    rates3=pd.DataFrame(rates2.iloc[:,-1])
    rates4=pd.DataFrame(rates2.iloc[:,0])
    rates5=pd.DataFrame(rates2.iloc[:,1])
    id=tx_bond_revamp['key'][i]

    rates3.columns = [rates2['key'][0]]
    rates4.columns = [id]
    rates5.columns = [id]
    interpolated_curve= pd.DataFrame(rates3)
    interpolated_curve_rate = pd.concat([interpolated_curve_rate,interpolated_curve], axis = 1)
    lw_rate1 = pd.concat([lw_rate1,rates4], axis = 1)
    up_rate1 = pd.concat([up_rate1,rates5], axis = 1)

In [88]:
spreads

,date,instrument,rate,rate_stressed
0,2019-12-31,IN1520190101NCD,5.443980,5.443980
1,2019-12-30,IN1520190101NCD,5.273412,5.273412
2,2019-12-27,IN1520190101NCD,5.299568,5.299568
3,2019-12-26,IN1520190101NCD,5.290016,5.290016
4,2019-12-24,IN1520190101NCD,5.510400,5.510400
...,...,...,...,...
1501,2018-12-26,INE134E08925PSUT,0.949293,0.949293
1502,2018-12-24,INE134E08925PSUT,0.949293,0.949293
1503,2018-12-21,INE134E08925PSUT,0.949293,0.949293
1504,2018-12-20,INE134E08925PSUT,0.949293,0.949293


In [89]:
##Unique instrument of tx_bond_revamp
aaa = tx_bond_revamp['instrument'].unique()
aaa = pd.DataFrame(aaa)
aaa


,0
0,IN0020100015CGS
1,IN1520190101NCD
2,IN1520190101PF


In [90]:
##Unique instrument of spreads
zzz = spreads['instrument'].unique()
zzz = pd.DataFrame(zzz)
zzz

,0
0,IN1520190101NCD
1,IN1520190101PF
2,IN1520190101RPF
3,INE036D08015SD
4,INE095H07014PSUTF
5,INE134E08925PSUT


In [91]:
##Finding common values between two dataframe
common = aaa.merge(zzz, on=[0])
common

,0
0,IN1520190101NCD
1,IN1520190101PF


In [92]:
common = list(common[0].values)
common

['IN1520190101NCD', 'IN1520190101PF']

In [93]:
#Matching the spreads and tx_bond dataframes on the basis of column 'instrument'
spreads1 = spreads.loc[spreads['instrument'].isin(common)]
spreads1

,date,instrument,rate,rate_stressed
0,2019-12-31,IN1520190101NCD,5.443980,5.443980
1,2019-12-30,IN1520190101NCD,5.273412,5.273412
2,2019-12-27,IN1520190101NCD,5.299568,5.299568
3,2019-12-26,IN1520190101NCD,5.290016,5.290016
4,2019-12-24,IN1520190101NCD,5.510400,5.510400
...,...,...,...,...
497,2018-12-26,IN1520190101PF,1.277626,1.277626
498,2018-12-24,IN1520190101PF,1.277626,1.277626
499,2018-12-21,IN1520190101PF,1.277626,1.277626
500,2018-12-20,IN1520190101PF,1.277626,1.277626


In [94]:
###True code

In [95]:
#instru_count = pd.DataFrame()
if len(spreads1) == 0:
    names = list(interpolated_curve_rate.columns)
    spreads_unstk = pd.DataFrame(columns = names)   
else:
    spread_1 = spreads1.loc[:, ['instrument','rate']]
    spread_1 = spread_1.set_index('instrument')
    spreads_unstk = (spread_1.assign(labels = spread_1.groupby(level = 0).cumcount())
                    .groupby([spread_1.index,'labels']).first()
                    .unstack('labels')
                    .sort_index(axis =1,level = 1)
                    .droplevel(1,axis = 1))
    spreads_unstk = spreads_unstk.transpose()
    spreads_unstk = spreads_unstk.iloc[:,[1,0]]
    instru_count = tx_bond_revamp.groupby('instrument').count()
    instru_count = pd.DataFrame(instru_count['k'])
    instru_count['instrument'] = instru_count.index
    instru_count = instru_count.reset_index(drop=True)
    cols = instru_count.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    instru_count = instru_count[cols]
    
    x = pd.DataFrame(spreads1['instrument'].unique())
    y = pd.DataFrame(instru_count['instrument'].unique())
    common1 = x.merge(y, on=[0])
    common1 = list(common1[0].values)
    freq = instru_count.loc[instru_count['instrument'].isin(common1)]
    
    for i in range(1, (len(freq['instrument'])+1)):
        freq['k'][i]
        a = freq['k'][i]-1
        for j in range (0, a):
            spreads_unstk = pd.concat([spreads_unstk,spreads_unstk.iloc[:,j]], axis = 1)

    #Sorting the spreads_unstk dataframe by column names
    spreads_unstk = spreads_unstk.sort_index(axis = 1)
    
    #Resetting the index of the dataframe
    spreads_unstk = spreads_unstk.reset_index(drop=True)
    
    #Creating an empty matrix
    spread_extend = np.zeros((n+1, len(tx_bond['instrument'])-len(spreads_unstk.columns)))
    spread_extend = pd.DataFrame(spread_extend)  
    
    #Concatenating spreads_unstk and spread_extend dataframes
    spreads_unstk = pd.concat([spread_extend,spreads_unstk],axis=1)
    
    #Renaming the column names of spreads_unstk by using the column names of interpolated_curve_rate
    spreads_unstk.columns =interpolated_curve_rate.columns
 

In [96]:
spreads_unstk

,IN0020100015CGS_1,IN0020100015CGS_2,IN0020100015CGS_3,IN0020100015CGS_4,IN0020100015CGS_5,IN0020100015CGS_6,IN1520190101NCD_1,IN1520190101NCD_2,IN1520190101NCD_3,IN1520190101NCD_4,IN1520190101NCD_5,IN1520190101PF_1,IN1520190101PF_2,IN1520190101PF_3,IN1520190101PF_4,IN1520190101PF_5,IN1520190101PF_6
0,0.0,0.0,0.0,0.0,0.0,0.0,5.443980,5.443980,5.443980,5.443980,5.443980,1.123651,1.123651,1.123651,1.123651,1.123651,1.123651
1,0.0,0.0,0.0,0.0,0.0,0.0,5.273412,5.273412,5.273412,5.273412,5.273412,1.264819,1.264819,1.264819,1.264819,1.264819,1.264819
2,0.0,0.0,0.0,0.0,0.0,0.0,5.299568,5.299568,5.299568,5.299568,5.299568,1.154296,1.154296,1.154296,1.154296,1.154296,1.154296
3,0.0,0.0,0.0,0.0,0.0,0.0,5.290016,5.290016,5.290016,5.290016,5.290016,1.272986,1.272986,1.272986,1.272986,1.272986,1.272986
4,0.0,0.0,0.0,0.0,0.0,0.0,5.510400,5.510400,5.510400,5.510400,5.510400,1.330279,1.330279,1.330279,1.330279,1.330279,1.330279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.0,0.0,0.0,0.0,0.0,0.0,5.443160,5.443160,5.443160,5.443160,5.443160,1.277626,1.277626,1.277626,1.277626,1.277626,1.277626
247,0.0,0.0,0.0,0.0,0.0,0.0,5.443160,5.443160,5.443160,5.443160,5.443160,1.277626,1.277626,1.277626,1.277626,1.277626,1.277626
248,0.0,0.0,0.0,0.0,0.0,0.0,5.443160,5.443160,5.443160,5.443160,5.443160,1.277626,1.277626,1.277626,1.277626,1.277626,1.277626
249,0.0,0.0,0.0,0.0,0.0,0.0,5.443160,5.443160,5.443160,5.443160,5.443160,1.277626,1.277626,1.277626,1.277626,1.277626,1.277626


In [97]:
#melt takes related columns with common values and bundles them into one column called ‘variable’. 
spreads_all = spreads_unstk.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)
spreads_all


,variable,value
0,IN0020100015CGS_1,0.000000
1,IN0020100015CGS_1,0.000000
2,IN0020100015CGS_1,0.000000
3,IN0020100015CGS_1,0.000000
4,IN0020100015CGS_1,0.000000
...,...,...
4262,IN1520190101PF_6,1.277626
4263,IN1520190101PF_6,1.277626
4264,IN1520190101PF_6,1.277626
4265,IN1520190101PF_6,1.277626


In [98]:
# using melt function on interpolated_curve_rate dataframe
interpolated_curve_rates = interpolated_curve_rate.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)
interpolated_curve_rates


,variable,value
0,IN0020100015CGS_1,5.060000
1,IN0020100015CGS_1,5.089231
2,IN0020100015CGS_1,5.098462
3,IN0020100015CGS_1,5.019231
4,IN0020100015CGS_1,5.045385
...,...,...
4262,IN1520190101PF_6,5.200000
4263,IN1520190101PF_6,5.140000
4264,IN1520190101PF_6,5.200000
4265,IN1520190101PF_6,5.190000


In [99]:
##Adding spreads_unstk and interpolated_curve_rate columns
final_ytm = spreads_unstk +interpolated_curve_rate
final_ytm

,IN0020100015CGS_1,IN0020100015CGS_2,IN0020100015CGS_3,IN0020100015CGS_4,IN0020100015CGS_5,IN0020100015CGS_6,IN1520190101NCD_1,IN1520190101NCD_2,IN1520190101NCD_3,IN1520190101NCD_4,IN1520190101NCD_5,IN1520190101PF_1,IN1520190101PF_2,IN1520190101PF_3,IN1520190101PF_4,IN1520190101PF_5,IN1520190101PF_6
0,5.060000,5.190000,5.331538,5.285543,5.776,5.776,10.521123,10.633980,11.219980,11.800980,11.800980,6.117151,6.250794,6.306912,6.313651,6.313651,6.313651
1,5.089231,5.160000,5.306154,5.251467,5.721,5.721,10.370555,10.433412,10.994412,11.943278,11.943278,6.303485,6.385038,6.418080,6.424819,6.424819,6.424819
2,5.098462,5.150000,5.297692,5.253859,5.787,5.787,10.413853,10.449568,11.086568,11.650790,11.650790,6.167130,6.314736,6.321144,6.304296,6.304296,6.304296
3,5.019231,5.200000,5.340000,5.299130,5.808,5.808,10.334302,10.490016,11.098016,11.642972,11.642972,6.242320,6.390349,6.466247,6.472986,6.472986,6.472986
4,5.045385,5.150000,5.297692,5.248641,5.755,5.755,10.568971,10.660400,11.265400,11.860178,11.860178,6.318946,6.427312,6.473540,6.480279,6.480279,6.480279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,5.063077,5.191630,5.341538,5.288533,5.743,5.743,10.526017,10.643160,11.186160,11.819115,11.819115,6.276292,6.418175,6.467626,6.474039,6.477626,6.477626
247,4.995385,5.181848,5.281538,5.251522,5.824,5.824,10.468874,10.583160,11.267160,11.735427,11.735427,6.258292,6.391802,6.467626,6.435560,6.417626,6.417626
248,5.056923,5.191630,5.341538,5.285272,5.723,5.723,10.514589,10.643160,11.166160,11.813338,11.813338,6.240126,6.391362,6.454147,6.474039,6.477626,6.477626
249,5.183846,5.190000,5.331538,5.288315,5.793,5.793,10.621731,10.633160,11.236160,11.798427,11.798427,6.379792,6.440812,6.454147,6.467626,6.467626,6.467626


In [100]:
#Transposing the final_ytm dataframe
t_final_ytm = final_ytm.T
t_final_ytm

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,250
IN0020100015CGS_1,5.060000,5.089231,5.098462,5.019231,5.045385,5.086923,5.040000,5.080000,5.053077,5.031538,...,5.040000,5.060769,4.940000,5.078462,5.117692,5.063077,4.995385,5.056923,5.183846,5.050000
IN0020100015CGS_2,5.190000,5.160000,5.150000,5.200000,5.150000,5.220000,5.130000,5.200000,5.200000,5.230000,...,5.186739,5.185109,5.191630,5.188370,5.191630,5.191630,5.181848,5.191630,5.190000,5.190000
IN0020100015CGS_3,5.331538,5.306154,5.297692,5.340000,5.297692,5.356923,5.280769,5.340000,5.340000,5.365385,...,5.311538,5.301538,5.341538,5.321538,5.341538,5.341538,5.281538,5.341538,5.331538,5.331538
IN0020100015CGS_4,5.285543,5.251467,5.253859,5.299130,5.248641,5.306087,5.228641,5.303533,5.299293,5.313315,...,5.261141,5.264022,5.288533,5.279130,5.294402,5.288533,5.251522,5.285272,5.288315,5.285543
IN0020100015CGS_5,5.776000,5.721000,5.787000,5.808000,5.755000,5.748000,5.735000,5.835000,5.809000,5.741000,...,5.729000,5.798000,5.743000,5.788000,5.779000,5.743000,5.824000,5.723000,5.793000,5.776000
IN0020100015CGS_6,5.776000,5.721000,5.787000,5.808000,5.755000,5.748000,5.735000,5.835000,5.809000,5.741000,...,5.729000,5.798000,5.743000,5.788000,5.779000,5.743000,5.824000,5.723000,5.793000,5.776000
IN1520190101NCD_1,10.521123,10.370555,10.413853,10.334302,10.568971,10.398586,10.503376,10.647692,10.636180,10.641914,...,10.500303,10.513160,10.417446,10.537446,10.567446,10.526017,10.468874,10.514589,10.621731,10.510303
IN1520190101NCD_2,10.633980,10.433412,10.449568,10.490016,10.660400,10.517158,10.576233,10.750549,10.763323,10.821914,...,10.613160,10.603160,10.643160,10.623160,10.643160,10.643160,10.583160,10.643160,10.633160,10.633160
IN1520190101NCD_3,11.219980,10.994412,11.086568,11.098016,11.265400,11.045158,11.181233,11.385549,11.372323,11.332914,...,11.172160,11.241160,11.186160,11.231160,11.222160,11.186160,11.267160,11.166160,11.236160,11.219160
IN1520190101NCD_4,11.800980,11.943278,11.650790,11.642972,11.860178,11.653580,11.804388,11.926616,11.921479,11.954114,...,11.780160,11.775360,11.810160,11.773693,11.813627,11.819115,11.735427,11.813338,11.798427,11.800160


In [101]:
t_final_ytm = t_final_ytm.eval('instrument = index')
cols1 = t_final_ytm.columns.tolist()
cols1 = cols1[-1:] + cols1[:-1]
t_final_ytm = t_final_ytm[cols1]
t_final_ytm

,instrument,0,1,2,3,4,5,6,7,8,...,241,242,243,244,245,246,247,248,249,250
IN0020100015CGS_1,IN0020100015CGS_1,5.060000,5.089231,5.098462,5.019231,5.045385,5.086923,5.040000,5.080000,5.053077,...,5.040000,5.060769,4.940000,5.078462,5.117692,5.063077,4.995385,5.056923,5.183846,5.050000
IN0020100015CGS_2,IN0020100015CGS_2,5.190000,5.160000,5.150000,5.200000,5.150000,5.220000,5.130000,5.200000,5.200000,...,5.186739,5.185109,5.191630,5.188370,5.191630,5.191630,5.181848,5.191630,5.190000,5.190000
IN0020100015CGS_3,IN0020100015CGS_3,5.331538,5.306154,5.297692,5.340000,5.297692,5.356923,5.280769,5.340000,5.340000,...,5.311538,5.301538,5.341538,5.321538,5.341538,5.341538,5.281538,5.341538,5.331538,5.331538
IN0020100015CGS_4,IN0020100015CGS_4,5.285543,5.251467,5.253859,5.299130,5.248641,5.306087,5.228641,5.303533,5.299293,...,5.261141,5.264022,5.288533,5.279130,5.294402,5.288533,5.251522,5.285272,5.288315,5.285543
IN0020100015CGS_5,IN0020100015CGS_5,5.776000,5.721000,5.787000,5.808000,5.755000,5.748000,5.735000,5.835000,5.809000,...,5.729000,5.798000,5.743000,5.788000,5.779000,5.743000,5.824000,5.723000,5.793000,5.776000
IN0020100015CGS_6,IN0020100015CGS_6,5.776000,5.721000,5.787000,5.808000,5.755000,5.748000,5.735000,5.835000,5.809000,...,5.729000,5.798000,5.743000,5.788000,5.779000,5.743000,5.824000,5.723000,5.793000,5.776000
IN1520190101NCD_1,IN1520190101NCD_1,10.521123,10.370555,10.413853,10.334302,10.568971,10.398586,10.503376,10.647692,10.636180,...,10.500303,10.513160,10.417446,10.537446,10.567446,10.526017,10.468874,10.514589,10.621731,10.510303
IN1520190101NCD_2,IN1520190101NCD_2,10.633980,10.433412,10.449568,10.490016,10.660400,10.517158,10.576233,10.750549,10.763323,...,10.613160,10.603160,10.643160,10.623160,10.643160,10.643160,10.583160,10.643160,10.633160,10.633160
IN1520190101NCD_3,IN1520190101NCD_3,11.219980,10.994412,11.086568,11.098016,11.265400,11.045158,11.181233,11.385549,11.372323,...,11.172160,11.241160,11.186160,11.231160,11.222160,11.186160,11.267160,11.166160,11.236160,11.219160
IN1520190101NCD_4,IN1520190101NCD_4,11.800980,11.943278,11.650790,11.642972,11.860178,11.653580,11.804388,11.926616,11.921479,...,11.780160,11.775360,11.810160,11.773693,11.813627,11.819115,11.735427,11.813338,11.798427,11.800160


In [102]:
# using melt function on t_final_ytm dataframe
t_final_ytm2 = t_final_ytm.melt(id_vars='instrument', value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)
t_final_ytm2

,instrument,variable,value
0,IN0020100015CGS_1,0,5.060000
1,IN0020100015CGS_2,0,5.190000
2,IN0020100015CGS_3,0,5.331538
3,IN0020100015CGS_4,0,5.285543
4,IN0020100015CGS_5,0,5.776000
...,...,...,...
4262,IN1520190101PF_2,250,6.394769
4263,IN1520190101PF_3,250,6.457517
4264,IN1520190101PF_4,250,6.467626
4265,IN1520190101PF_5,250,6.467626


In [103]:
#To sort the dataframe instrument and variable wise
t_final_ytm2 = t_final_ytm2.sort_values(['instrument', 'variable'], ascending=[True, True])
t_final_ytm2

,instrument,variable,value
0,IN0020100015CGS_1,0,5.060000
17,IN0020100015CGS_1,1,5.089231
34,IN0020100015CGS_1,2,5.098462
51,IN0020100015CGS_1,3,5.019231
68,IN0020100015CGS_1,4,5.045385
...,...,...,...
4198,IN1520190101PF_6,246,6.477626
4215,IN1520190101PF_6,247,6.417626
4232,IN1520190101PF_6,248,6.477626
4249,IN1520190101PF_6,249,6.467626


In [104]:
#Renaming instrument column as key
t_final_ytm2.rename({'instrument': 'key'}, axis=1, inplace=True)

In [105]:
 t_final_ytm2

,key,variable,value
0,IN0020100015CGS_1,0,5.060000
17,IN0020100015CGS_1,1,5.089231
34,IN0020100015CGS_1,2,5.098462
51,IN0020100015CGS_1,3,5.019231
68,IN0020100015CGS_1,4,5.045385
...,...,...,...
4198,IN1520190101PF_6,246,6.477626
4215,IN1520190101PF_6,247,6.417626
4232,IN1520190101PF_6,248,6.477626
4249,IN1520190101PF_6,249,6.467626


In [106]:
#Merge two dataframe that is tx_bond_revamp and t_final_ytm2
t_final_ytm_merge = pd.merge(tx_bond_revamp, t_final_ytm2, on='key')

In [107]:
t_final_ytm_merge

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm,variable,value
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.17,5.06,0.000000,5.060000,105,105,365,3943.601917,0,5.060000
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.17,5.06,0.000000,5.060000,105,105,365,3943.601917,1,5.089231
2,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.17,5.06,0.000000,5.060000,105,105,365,3943.601917,2,5.098462
3,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.17,5.06,0.000000,5.060000,105,105,365,3943.601917,3,5.019231
4,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.17,5.06,0.000000,5.060000,105,105,365,3943.601917,4,5.045385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,5.19,1.123651,6.313651,425,426,365,186207.562886,246,6.477626
4263,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,5.19,1.123651,6.313651,425,426,365,186207.562886,247,6.417626
4264,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,5.19,1.123651,6.313651,425,426,365,186207.562886,248,6.477626
4265,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,5.19,1.123651,6.313651,425,426,365,186207.562886,249,6.467626


In [108]:
t_final_ytm_merge['interpolated_curve_rates'] = interpolated_curve_rates['value']
t_final_ytm_merge

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm,variable,value,interpolated_curve_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,0,5.060000,5.060000
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,1,5.089231,5.089231
2,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,2,5.098462,5.098462
3,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,3,5.019231,5.019231
4,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,4,5.045385,5.045385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,1.123651,6.313651,425,426,365,186207.562886,246,6.477626,5.200000
4263,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,1.123651,6.313651,425,426,365,186207.562886,247,6.417626,5.140000
4264,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,1.123651,6.313651,425,426,365,186207.562886,248,6.477626,5.200000
4265,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,1.123651,6.313651,425,426,365,186207.562886,249,6.467626,5.190000


In [109]:
t_final_ytm_merge['value'] = t_final_ytm_merge['value'] + t_final_ytm_merge['mark_up_value']
t_final_ytm_merge

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm,variable,value,interpolated_curve_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,0,5.060000,5.060000
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,1,5.089231,5.089231
2,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,2,5.098462,5.098462
3,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,3,5.019231,5.019231
4,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.06,0.000000,5.060000,105,105,365,3943.601917,4,5.045385,5.045385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,1.123651,6.313651,425,426,365,186207.562886,246,6.477626,5.200000
4263,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,1.123651,6.313651,425,426,365,186207.562886,247,6.417626,5.140000
4264,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,1.123651,6.313651,425,426,365,186207.562886,248,6.477626,5.200000
4265,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,5.19,1.123651,6.313651,425,426,365,186207.562886,249,6.467626,5.190000


In [110]:
t_final_ytm_merge['spreads_all'] = spreads_all['value']
t_final_ytm_merge

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm,variable,value,interpolated_curve_rates,spreads_all
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0.000000,5.060000,105,105,365,3943.601917,0,5.060000,5.060000,0.000000
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0.000000,5.060000,105,105,365,3943.601917,1,5.089231,5.089231,0.000000
2,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0.000000,5.060000,105,105,365,3943.601917,2,5.098462,5.098462,0.000000
3,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0.000000,5.060000,105,105,365,3943.601917,3,5.019231,5.019231,0.000000
4,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0.000000,5.060000,105,105,365,3943.601917,4,5.045385,5.045385,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,1.123651,6.313651,425,426,365,186207.562886,246,6.477626,5.200000,1.277626
4263,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,1.123651,6.313651,425,426,365,186207.562886,247,6.417626,5.140000,1.277626
4264,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,1.123651,6.313651,425,426,365,186207.562886,248,6.477626,5.200000,1.277626
4265,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,1.123651,6.313651,425,426,365,186207.562886,249,6.467626,5.190000,1.277626


In [111]:
##To generate index by considering particular category which has a sequence from 0 to 250 and repeats itself as per the shape of the dataframe

In [112]:
new_res_df = pd.DataFrame()
for a, g in t_final_ytm_merge.groupby(["key"]):
    new_df = g.reset_index(drop = True)
    new_res_df = new_res_df.append(new_df)

/tmp/ipykernel_316/356069128.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_res_df = new_res_df.append(new_df)
/tmp/ipykernel_316/356069128.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_res_df = new_res_df.append(new_df)
/tmp/ipykernel_316/356069128.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_res_df = new_res_df.append(new_df)
/tmp/ipykernel_316/356069128.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_res_df = new_res_df.append(new_df)
/tmp/ipykernel_316/356069128.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [113]:
t_final_ytm_merge_new  = new_res_df.reset_index(drop = False)

In [114]:
t_final_ytm_merge_new

,index,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,...,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm,variable,value,interpolated_curve_rates,spreads_all
0,0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,0,5.060000,5.060000,0.000000
1,1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,1,5.089231,5.089231,0.000000
2,2,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,2,5.098462,5.098462,0.000000
3,3,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,3,5.019231,5.019231,0.000000
4,4,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,4,5.045385,5.045385,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,246,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,...,1.123651,6.313651,425,426,365,186207.562886,246,6.477626,5.200000,1.277626
4263,247,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,...,1.123651,6.313651,425,426,365,186207.562886,247,6.417626,5.140000,1.277626
4264,248,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,...,1.123651,6.313651,425,426,365,186207.562886,248,6.477626,5.200000,1.277626
4265,249,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,...,1.123651,6.313651,425,426,365,186207.562886,249,6.467626,5.190000,1.277626


In [115]:
t_final_ytm_merge_new.rename(columns = {'index':'env'}, inplace = True)


In [116]:
t_final_ytm_merge_new

,env,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,...,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm,variable,value,interpolated_curve_rates,spreads_all
0,0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,0,5.060000,5.060000,0.000000
1,1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,1,5.089231,5.089231,0.000000
2,2,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,2,5.098462,5.098462,0.000000
3,3,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,3,5.019231,5.019231,0.000000
4,4,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,...,0.000000,5.060000,105,105,365,3943.601917,4,5.045385,5.045385,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,246,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,...,1.123651,6.313651,425,426,365,186207.562886,246,6.477626,5.200000,1.277626
4263,247,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,...,1.123651,6.313651,425,426,365,186207.562886,247,6.417626,5.140000,1.277626
4264,248,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,...,1.123651,6.313651,425,426,365,186207.562886,248,6.477626,5.200000,1.277626
4265,249,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,...,1.123651,6.313651,425,426,365,186207.562886,249,6.467626,5.190000,1.277626


In [117]:
# Move first column to the Last for better arrangement purpose
temp_cols=t_final_ytm_merge_new.columns.tolist()
new_cols=temp_cols[1:] + temp_cols[0:1]
t_final_ytm_merge_new=t_final_ytm_merge_new[new_cols]
t_final_ytm_merge_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm,variable,value,interpolated_curve_rates,spreads_all,env
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.060000,105,105,365,3943.601917,0,5.060000,5.060000,0.000000,0
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.060000,105,105,365,3943.601917,1,5.089231,5.089231,0.000000,1
2,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.060000,105,105,365,3943.601917,2,5.098462,5.098462,0.000000,2
3,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.060000,105,105,365,3943.601917,3,5.019231,5.019231,0.000000,3
4,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,5.060000,105,105,365,3943.601917,4,5.045385,5.045385,0.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,6.313651,425,426,365,186207.562886,246,6.477626,5.200000,1.277626,246
4263,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,6.313651,425,426,365,186207.562886,247,6.417626,5.140000,1.277626,247
4264,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,6.313651,425,426,365,186207.562886,248,6.477626,5.200000,1.277626,248
4265,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,6.313651,425,426,365,186207.562886,249,6.467626,5.190000,1.277626,249


# All_MTM


In [118]:
t_final_ytm_merge_new['all_mtm'] =t_final_ytm_merge_new['Cashflow'] / ((1+(t_final_ytm_merge_new['value']/100))**(t_final_ytm_merge_new['residual_maturity_Actual']/t_final_ytm_merge_new['no_of_days_in_a_year']))
t_final_ytm_merge_new

/tmp/ipykernel_316/4160071924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_final_ytm_merge_new['all_mtm'] =t_final_ytm_merge_new['Cashflow'] / ((1+(t_final_ytm_merge_new['value']/100))**(t_final_ytm_merge_new['residual_maturity_Actual']/t_final_ytm_merge_new['no_of_days_in_a_year']))


,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm,variable,value,interpolated_curve_rates,spreads_all,env,all_mtm
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,105,105,365,3943.601917,0,5.060000,5.060000,0.000000,0,3943.601917
1,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,105,105,365,3943.601917,1,5.089231,5.089231,0.000000,1,3943.286333
2,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,105,105,365,3943.601917,2,5.098462,5.098462,0.000000,2,3943.186699
3,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,105,105,365,3943.601917,3,5.019231,5.019231,0.000000,3,3944.042262
4,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,105,105,365,3943.601917,4,5.045385,5.045385,0.000000,4,3943.759751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,425,426,365,186207.562886,246,6.477626,5.200000,1.277626,246,185872.923952
4263,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,425,426,365,186207.562886,247,6.417626,5.140000,1.277626,247,185995.242132
4264,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,425,426,365,186207.562886,248,6.477626,5.200000,1.277626,248,185872.923952
4265,PF001,IN1520190101PF,16803,6,INR,INR,200000,P,200000,100,...,425,426,365,186207.562886,249,6.467626,5.190000,1.277626,249,185893.299942


In [119]:
#Converting MTM dataframe to csv file
t_final_ytm_merge_new.to_csv('Result_mtm.csv')

In [120]:
##To fetch particular columns from the dataframe and storing it in a dataframe that is 'Result'

In [121]:
Result = t_final_ytm_merge_new[['env', 'instrument','key','k','Cashflow','Cashflow_Type','Face_value','YTM','cashflow_mtm','value','all_mtm']]

In [122]:
Result

,env,instrument,key,k,Cashflow,Cashflow_Type,Face_value,YTM,cashflow_mtm,value,all_mtm
0,0,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.060000,3943.601917
1,1,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.089231,3943.286333
2,2,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.098462,3943.186699
3,3,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.019231,3944.042262
4,4,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.045385,3943.759751
...,...,...,...,...,...,...,...,...,...,...,...
4262,246,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952
4263,247,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.417626,185995.242132
4264,248,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952
4265,249,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.467626,185893.299942


# CHANGE_IN_MTM


In [123]:
##Concern: This is a R code where other combinations of functions are used . Here I have simple computed change_in_mtm by subtracting the particular columns 
#result_1 <- ddply(Result, .(key,instrument,env), mutate, change_in_mtm = all_mtm - cashflow_mtm)


In [124]:
Result['change_in_mtm'] = Result['all_mtm'] - Result['cashflow_mtm']

/tmp/ipykernel_316/1636623134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Result['change_in_mtm'] = Result['all_mtm'] - Result['cashflow_mtm']


In [125]:
Result

,env,instrument,key,k,Cashflow,Cashflow_Type,Face_value,YTM,cashflow_mtm,value,all_mtm,change_in_mtm
0,0,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.060000,3943.601917,0.0
1,1,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.089231,3943.286333,-0.315584
2,2,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.098462,3943.186699,-0.415218
3,3,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.019231,3944.042262,0.440345
4,4,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.045385,3943.759751,0.157834
...,...,...,...,...,...,...,...,...,...,...,...,...
4262,246,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952,-334.638934
4263,247,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.417626,185995.242132,-212.320755
4264,248,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952,-334.638934
4265,249,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.467626,185893.299942,-314.262945


In [126]:
#Generating key2
Result["key2"] = Result['instrument'].map(str) + '_' + Result['env'].map(str) 
Result

/tmp/ipykernel_316/1336550204.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Result["key2"] = Result['instrument'].map(str) + '_' + Result['env'].map(str)


,env,instrument,key,k,Cashflow,Cashflow_Type,Face_value,YTM,cashflow_mtm,value,all_mtm,change_in_mtm,key2
0,0,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.060000,3943.601917,0.0,IN0020100015CGS_0
1,1,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.089231,3943.286333,-0.315584,IN0020100015CGS_1
2,2,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.098462,3943.186699,-0.415218,IN0020100015CGS_2
3,3,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.019231,3944.042262,0.440345,IN0020100015CGS_3
4,4,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.045385,3943.759751,0.157834,IN0020100015CGS_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,246,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952,-334.638934,IN1520190101PF_246
4263,247,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.417626,185995.242132,-212.320755,IN1520190101PF_247
4264,248,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952,-334.638934,IN1520190101PF_248
4265,249,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.467626,185893.299942,-314.262945,IN1520190101PF_249


# TOTAL CHANGE IN MTM


In [127]:
#Grouping by two categories that is by variable 'instrument' and 'env' and finding sum over 'change_in_mtm' by considering particular vertices

In [128]:
total_change_in_MTM = pd.DataFrame(Result.groupby(["instrument", "env"])["change_in_mtm"].sum())

In [129]:
total_change_in_MTM

change_in_mtm
instrument      env              
IN0020100015CGS 0             0.0
                1      112.718881
                2      -17.497122
                3      -64.347071
                4       46.709385
...                           ...
IN1520190101PF  246   -358.859981
                247   -231.269147
                248   -357.653466
                249    -338.56027
                250   -336.840118

[753 rows x 1 columns]

In [130]:
total_change_in_MTM  = total_change_in_MTM.reset_index(drop = False)

In [131]:
total_change_in_MTM

,instrument,env,change_in_mtm
0,IN0020100015CGS,0,0.0
1,IN0020100015CGS,1,112.718881
2,IN0020100015CGS,2,-17.497122
3,IN0020100015CGS,3,-64.347071
4,IN0020100015CGS,4,46.709385
...,...,...,...
748,IN1520190101PF,246,-358.859981
749,IN1520190101PF,247,-231.269147
750,IN1520190101PF,248,-357.653466
751,IN1520190101PF,249,-338.56027


In [132]:
total_change_in_MTM_1 = total_change_in_MTM.sort_values(by=['env'])

In [133]:
total_change_in_MTM_1.reset_index(drop=True)

,instrument,env,change_in_mtm
0,IN0020100015CGS,0,0.0
1,IN1520190101NCD,0,0.0
2,IN1520190101PF,0,0.0
3,IN0020100015CGS,1,112.718881
4,IN1520190101NCD,1,-397.576788
...,...,...,...
748,IN1520190101NCD,249,1.631663
749,IN0020100015CGS,249,-35.127303
750,IN0020100015CGS,250,0.107989
751,IN1520190101NCD,250,3.44085


In [134]:
##Sorting the dataframe by instrument and env wise
total_change_in_MTM_1 = total_change_in_MTM.sort_values(by=["instrument","env"])

In [135]:
total_change_in_MTM_1

,instrument,env,change_in_mtm
0,IN0020100015CGS,0,0.0
1,IN0020100015CGS,1,112.718881
2,IN0020100015CGS,2,-17.497122
3,IN0020100015CGS,3,-64.347071
4,IN0020100015CGS,4,46.709385
...,...,...,...
748,IN1520190101PF,246,-358.859981
749,IN1520190101PF,247,-231.269147
750,IN1520190101PF,248,-357.653466
751,IN1520190101PF,249,-338.56027


# PORTFOLIO TOTAL CHANGE IN MTM

In [136]:
#Grouping by env and summimg over change_in_mtm

In [137]:
portfolio_change_in_MTM = total_change_in_MTM_1.groupby('env')['change_in_mtm'].sum()

In [138]:
portfolio_change_in_MTM

env
0             0.0
1     -529.463523
2       564.43744
3      172.884205
4     -532.535005
          ...    
246   -349.611674
247   -100.980956
248   -284.457989
249    -372.05591
250   -333.291279
Name: change_in_mtm, Length: 251, dtype: object

In [139]:
portfolio_change_in_mtm = list(portfolio_change_in_MTM)
portfolio_change_in_mtm.pop(0) ##Removing the first value that is 0
portfolio_change_in_mtm

[-529.4635229149208,
 564.4374403301422,
 172.8842047604876,
 -532.535004526706,
 514.9047354323916,
 -15.16529680425674,
 -727.777668605982,
 -952.1703820309076,
 -1098.7530848278388,
 -824.6884610681896,
 -1077.1030860004862,
 -1095.5568525462422,
 -1004.3648987088168,
 -780.7648404164174,
 -411.2607502292931,
 -286.34203113430885,
 -704.9388596645872,
 -573.4835194635584,
 -849.7782010986871,
 -769.6832605790805,
 529.0953662875268,
 -721.549127363005,
 -363.6219044389104,
 -660.464078680603,
 -319.87177001199734,
 -1059.9208550329035,
 -1066.0998618607464,
 -332.08341103744124,
 -583.5910163714661,
 -252.8185066724218,
 -1024.068751885654,
 -830.5363109164132,
 -62.45866703210868,
 -1095.7345053527906,
 344.18142342599276,
 -668.762126467172,
 394.27378656787687,
 -549.0528120152808,
 415.69124923823256,
 514.8259676207886,
 572.0388837828914,
 470.388836317567,
 -985.8309593864487,
 -1032.5325501205652,
 -952.549620538814,
 -752.0198260457446,
 -750.1482698997152,
 -438.7100119489

In [140]:
len(portfolio_change_in_mtm)

250

In [141]:
portfolio_change_in_mtm

[-529.4635229149208,
 564.4374403301422,
 172.8842047604876,
 -532.535004526706,
 514.9047354323916,
 -15.16529680425674,
 -727.777668605982,
 -952.1703820309076,
 -1098.7530848278388,
 -824.6884610681896,
 -1077.1030860004862,
 -1095.5568525462422,
 -1004.3648987088168,
 -780.7648404164174,
 -411.2607502292931,
 -286.34203113430885,
 -704.9388596645872,
 -573.4835194635584,
 -849.7782010986871,
 -769.6832605790805,
 529.0953662875268,
 -721.549127363005,
 -363.6219044389104,
 -660.464078680603,
 -319.87177001199734,
 -1059.9208550329035,
 -1066.0998618607464,
 -332.08341103744124,
 -583.5910163714661,
 -252.8185066724218,
 -1024.068751885654,
 -830.5363109164132,
 -62.45866703210868,
 -1095.7345053527906,
 344.18142342599276,
 -668.762126467172,
 394.27378656787687,
 -549.0528120152808,
 415.69124923823256,
 514.8259676207886,
 572.0388837828914,
 470.388836317567,
 -985.8309593864487,
 -1032.5325501205652,
 -952.549620538814,
 -752.0198260457446,
 -750.1482698997152,
 -438.7100119489

In [142]:
instruments = list(total_change_in_MTM['instrument'].unique())
instruments

['IN0020100015CGS', 'IN1520190101NCD', 'IN1520190101PF']

In [143]:
Result

,env,instrument,key,k,Cashflow,Cashflow_Type,Face_value,YTM,cashflow_mtm,value,all_mtm,change_in_mtm,key2
0,0,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.060000,3943.601917,0.0,IN0020100015CGS_0
1,1,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.089231,3943.286333,-0.315584,IN0020100015CGS_1
2,2,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.098462,3943.186699,-0.415218,IN0020100015CGS_2
3,3,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.019231,3944.042262,0.440345,IN0020100015CGS_3
4,4,IN0020100015CGS,IN0020100015CGS_1,1,4000,C,100,5.060000,3943.601917,5.045385,3943.759751,0.157834,IN0020100015CGS_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,246,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952,-334.638934,IN1520190101PF_246
4263,247,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.417626,185995.242132,-212.320755,IN1520190101PF_247
4264,248,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952,-334.638934,IN1520190101PF_248
4265,249,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.467626,185893.299942,-314.262945,IN1520190101PF_249


In [196]:
decay_factor = 0.94

In [195]:
exact = []
near = []
decay_weights = []
instruments = list(total_change_in_MTM['instrument'].unique())
result_instr = list(Result['instrument'].unique())
prob = 1-confidence_level
for i in range(0,len(instruments)):
            Bondwise_VaR_1 = loadData(total_change_in_MTM,'instrument',types = instruments[i])
            Change_in_MTM = list(Bondwise_VaR_1['change_in_mtm'])
            Change_in_MTM.pop(0)
            Change_in_mtm_sorted = sorted(Change_in_MTM, reverse = True)
            if decay_factor == 0:
                    bondwise_VaR_exact_one_day = -(np.quantile(Change_in_mtm_sorted, prob))
                    exact.append(bondwise_VaR_exact_one_day)
                    nearby = np.floor(confidence_level * len(Change_in_mtm_sorted))
                    nearby = int(nearby)
                    nearby_nxt = nearby + 1

                    diff1 = pd.to_numeric(bondwise_VaR_exact_one_day + Change_in_mtm_sorted[nearby])
                    diff2 = pd.to_numeric(- Change_in_mtm_sorted[nearby_nxt] - bondwise_VaR_exact_one_day)

                    if (diff1 == diff2):
                        bondwise_VaR_nearby_one_day = - Change_in_mtm_sorted[nearby + 1]
                        near.append(bondwise_VaR_nearby_one_day)
                    elif (diff1 < diff2):
                        bondwise_VaR_nearby_one_day = - Change_in_mtm_sorted[nearby]
                        near.append(bondwise_VaR_nearby_one_day)
                    else:
                        bondwise_VaR_nearby_one_day = - Change_in_mtm_sorted[nearby + 1]
                        near.append(bondwise_VaR_nearby_one_day)
            
            #Bondwise_VaR_2 = loadData(Result,'instrument',types = instruments[i])
            
            else:
                a = []
                a = list((Bondwise_VaR_2['env'].unique()))
                a.pop()
                for number in a:
                    decay_weights.append((1-decay_factor)*(decay_factor ** number))

                total_mtm = Result.groupby('env')[['all_mtm']].sum
                #a= pd.DataFrame(range(0,(len(['env'].unique())-2)))
                #hp['decay_factor'].pow(a)
                #decay_weights = (1-decay_factor) * (hp['decay_factor'].pow(a))/((1 - decay_factor) ** (len(Result['env'].unique() -1)))
                total_mtm = Result.groupby('env')['all_mtm'].sum()
                total_mtm = list(total_mtm)
                mtm_returns = []
                
                for i in range(1,(len(Result['env'].unique() -1))):
                    fv = total_mtm[0]
                    returns = (total_mtm[i] - fv)/fv
                    mtm_returns.append(returns)
                    
                #decay_weights.rename(columns = {0:'decay_weights'}, inplace = True)
                decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
                mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
                mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
                mtm = mtm.sort_values(['mtm_returns']).reset_index()
                mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
                alpha = 1-confidence_level
                mtm1 = mtm.loc[mtm['cum_decay_weights'] < alpha]

######################################################################################################
    
                if (len(mtm1['cum_decay_weights'])==0):
                        bondwise_VaR_exact_one_day = np.abs(- mtm['mtm_returns'][0] * total_mtm[0])
                        exact.append(bondwise_VaR_exact_one_day)



                        bondwise_VaR_nearby_one_day = bondwise_VaR_exact_one_day
                        near.append(bondwise_VaR_nearby_one_day)



                else:
                        index = len(mtm1['cum_decay_weights'])

                        interpolated_return_exact = mtm['mtm_returns'][index-1] + ((mtm['mtm_returns'][index] - mtm['mtm_returns'][index-1]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) / (mtm['cum_decay_weights'][index] - mtm['cum_decay_weights'][index-1]))

                        bondwise_VaR_exact_one_day =  abs((total_mtm[0] * (interpolated_return_exact)))
                        exact.append(bondwise_VaR_exact_one_day)

                        if((((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) < (mtm['cum_decay_weights'][index] - (1 - confidence_level) ))):

                                        interpolated_return_nearby = mtm['mtm_returns'][index-1]
                                        bondwise_VaR_nearby_one_day = abs((total_mtm[0] * (interpolated_return_nearby)))
                                        near.append(bondwise_VaR_nearby_one_day)
                                        

                        else:
                                        index = len(mtm1['cum_decay_weights'])
                                        interpolated_return_nearby = mtm['mtm_returns'][index]
                                        VaR_nearby_one_day = abs((total_mtm[0] * (interpolated_return_nearby)))
                                        near.append(bondwise_VaR_nearby_one_day)



/tmp/ipykernel_316/1374054508.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  decay_weights.append((1-decay_factor)*(decay_factor ** number))


TypeError: cannot concatenate object of type '<class 'numpy.float64'>'; only Series and DataFrame objs are valid

In [198]:
 decay_factor ** number

1.0

In [199]:
1-decay_factor

0.06000000000000005

In [183]:
#Bondwise_VaR_2 = loadData(Result,'instrument',types = instruments[1])
prob

0.010000000000000009

In [184]:
result_instr = list(Result['instrument'].unique())
result_instr

['IN0020100015CGS', 'IN1520190101NCD', 'IN1520190101PF']

In [185]:
instruments = list(total_change_in_MTM['instrument'].unique())
instruments

['IN0020100015CGS', 'IN1520190101NCD', 'IN1520190101PF']

In [186]:
exact

[154.3716849931405, 569.8660218144913, 662.1777980284108]

In [187]:
near

[177.37545823556184, 572.2799557685521, 672.3680768321956]

In [167]:
Bondwise_VaR_2

,env,instrument,key,k,Cashflow,Cashflow_Type,Face_value,YTM,cashflow_mtm,value,all_mtm,change_in_mtm,key2
0,0,IN1520190101PF,IN1520190101PF_1,1,5000,C,100,6.117151,4949.826628,6.117151,4949.826628,0.0,IN1520190101PF_0
1,1,IN1520190101PF,IN1520190101PF_1,1,5000,C,100,6.117151,4949.826628,6.303485,4948.35177,-1.474858,IN1520190101PF_1
2,2,IN1520190101PF,IN1520190101PF_1,1,5000,C,100,6.117151,4949.826628,6.167130,4949.430745,-0.395883,IN1520190101PF_2
3,3,IN1520190101PF,IN1520190101PF_1,1,5000,C,100,6.117151,4949.826628,6.242320,4948.83557,-0.991058,IN1520190101PF_3
4,4,IN1520190101PF,IN1520190101PF_1,1,5000,C,100,6.117151,4949.826628,6.318946,4948.229532,-1.597096,IN1520190101PF_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,246,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952,-334.638934,IN1520190101PF_246
1502,247,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.417626,185995.242132,-212.320755,IN1520190101PF_247
1503,248,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.477626,185872.923952,-334.638934,IN1520190101PF_248
1504,249,IN1520190101PF,IN1520190101PF_6,6,200000,P,100,6.313651,186207.562886,6.467626,185893.299942,-314.262945,IN1520190101PF_249


In [ ]:
Change_in_MTM

In [ ]:
Bondwise_VaR

# VaR Computations for PORTFOLIO

In [ ]:
hp

# Computing VaR when df = 0 and hp = 1

In [ ]:
hp

In [ ]:
decay_factor=0

In [ ]:
##Corrected code 
prob = (1 - confidence_level)
decay_weights = []
if (decay_factor==0):
    VaR_exact_one_day = -(np.quantile(portfolio_change_in_mtm, prob))
    Portfolio_change_in_MTM = sorted(portfolio_change_in_mtm, reverse = True)
    nearby = np.floor(confidence_level * len(Portfolio_change_in_MTM))
    nearby = int(nearby)
    nearby_nxt = nearby + 1
    
    diff1 = pd.to_numeric(VaR_exact_one_day + Portfolio_change_in_MTM[nearby])
    diff2 = pd.to_numeric(- Portfolio_change_in_MTM[nearby_nxt] - VaR_exact_one_day)
                          
    if (diff1 == diff2):
        VaR_nearby_one_day = - Portfolio_change_in_MTM[nearby + 1]
    elif (diff1 < diff2):
        VaR_nearby_one_day = - Portfolio_change_in_MTM[nearby]
    else:
        VaR_nearby_one_day = - Portfolio_change_in_MTM[nearby + 1]



else:
    a = []
    a = list((Result['env'].unique()))
    a.pop()
    for number in a:
        decay_weights.append((1-decay_factor)*(decay_factor ** number))
    
    total_mtm = Result.groupby('env')[['all_mtm']].sum
    a= pd.DataFrame(range(0,(len(Result['env'].unique())-2)))
    #hp['decay_factor'].pow(a)
    #decay_weights = (1-decay_factor) * (hp['decay_factor'].pow(a))/((1 - decay_factor) ** (len(Result['env'].unique() -1)))
    total_mtm = Result.groupby('env')['all_mtm'].sum()
    total_mtm = list(total_mtm)
    mtm_returns = []
    
    for i in range(1,(len(Result['env'].unique() -1))):
                   fv = total_mtm[0]
                   returns = (total_mtm[i] - fv)/fv
                   mtm_returns.append(returns)
    #decay_weights.rename(columns = {0:'decay_weights'}, inplace = True)
    decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
    mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
    mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
    mtm = mtm.sort_values(['mtm_returns']).reset_index()
    mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
    alpha = 1-confidence_level
    mtm1 = mtm.loc[mtm['cum_decay_weights'] < alpha]
    
######################################################################################################
    
    if (len(mtm1['cum_decay_weights'])==0):
                  VaR_exact_one_day = np.abs(- mtm['mtm_returns'][0] * total_mtm['mtm_bc'][0])



                  VaR_exact_one_day = VaR_exact_one_day



    else:
                index = len(mtm1['cum_decay_weights'])
                                         
                interpolated_return_exact = mtm['mtm_returns'][index-1] + ((mtm['mtm_returns'][index] - mtm['mtm_returns'][index-1]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) / (mtm['cum_decay_weights'][index] - mtm['cum_decay_weights'][index-1]))
    
                VaR_exact_one_day =  abs((total_mtm[0] * (interpolated_return_exact)))
    
                if((((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) < (mtm['cum_decay_weights'][index] - (1 - confidence_level) ))):
            
                            interpolated_return_nearby = mtm['mtm_returns'][index-1]
                            VAR_nearby_one_day = abs((total_mtm[0] * (interpolated_return_nearby))) 
                             
                else:
                            index = len(mtm1['cum_decay_weights'])
                            print(index)
                            interpolated_return_nearby = mtm['mtm_returns'][index]
                            print(interpolated_return_nearby)
                            VAR_nearby_one_day = abs((total_mtm[0] * (interpolated_return_nearby)))
                            print(VAR_nearby_one_day)

In [ ]:
Bondwise_VAR = total_change_in_MTM.groupby('instrument')

In [ ]:
VaR_exact_one_day

In [ ]:
VaR_nearby_one_day

## Bondwise VAR

In [ ]:
##Sorting the dataframe in descending order
trial_total = pd.DataFrame()
for aa, gg in total_change_in_MTM.groupby("instrument"):
    gg = gg.reset_index(drop = True)
    gg = gg[gg.change_in_mtm != 0.0]
    gg = gg.sort_values("change_in_mtm",ascending=False, ignore_index=True)
    trial_total = trial_total.append(gg)

In [ ]:
trial_total

# Holding Period 10

In [ ]:
rates_interest_hp = getData(path8)

In [ ]:
holding_period = 10

In [ ]:
rates_interest_hp["key"] = rates_interest_hp["curve"] + '_' + rates_interest_hp["tenor"]

def sim_returns(df):
        simple_returns = pd.DataFrame([])

        for a, g in df.groupby('key'):

            g = g.reset_index(drop = True)
            
            new_row = g.iloc[0].to_dict()
            new_row['simple_returns'] = [0]
            top_row = pd.DataFrame(new_row)
            g = g.reset_index(drop = True)
            g['simple_returns'] = g['rate']-g['rate'].shift(-holding_period)
            g = pd.concat([top_row, g]).reset_index(drop = True)
            g['simulated rates'] = g['simple_returns'].apply(lambda x: np.exp(x)*j["rate"].iloc[0])
            #g['simulated rates'] =( g['simple_returns'] + g['rate'].iloc[0]).shift(fill_value = g['rate'][0])
            #g = g.iloc[:-1 , :]
            #g = pd.concat([top_row, g]).reset_index(drop = True)
            simple_returns = simple_returns.append(g)

    
        return simple_returns

In [ ]:
simulated_interest_hp = sim_returns(rates_interest_hp)

In [ ]:
g

In [ ]:
simulated_interest_hp

In [ ]:
rates_interest_hp

In [200]:
a = []
a = list((Bondwise_VaR_2['env'].unique()))
a.pop()
for number in a:
    decay_weights.append((1-decay_factor)*(decay_factor ** number))

/tmp/ipykernel_316/3780042832.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  decay_weights.append((1-decay_factor)*(decay_factor ** number))


TypeError: cannot concatenate object of type '<class 'numpy.float64'>'; only Series and DataFrame objs are valid

Most correct till above 

# Instrument-Wise VaR Computations


In [ ]:
###VaR_Exact computation for single instrument

In [ ]:
##Sample sorting function
total_change_in_MTM

In [ ]:
total_change_in_MTM  = total_change_in_MTM.reset_index(drop = False)
total_change_in_MTM

In [ ]:
total_change_in_MTM = total_change_in_MTM.sort_values(by=['env'])
total_change_in_MTM.reset_index(drop=True)

In [ ]:
#Remove rows with env=0

In [ ]:
Bondwise_change_in_MTM = total_change_in_MTM.iloc[3:]
Bondwise_change_in_MTM.reset_index(drop=True)

In [ ]:
###Considering a single instrument
Bondwise_change_in_MTM = Bondwise_change_in_MTM.loc[Bondwise_change_in_MTM['instrument'] == 'IN0020100015CGS']
Bondwise_change_in_MTM

In [ ]:
##Sorting in descending order
Bondwise_change_in_MTM = Bondwise_change_in_MTM.sort_values(by=['change_in_mtm'], ascending=False)
Bondwise_change_in_MTM.reset_index(drop = True)

In [ ]:
#Drop second column
Bondwise_change_in_MTM = Bondwise_change_in_MTM.drop(Bondwise_change_in_MTM.columns[1],axis = 1)
Bondwise_change_in_MTM.reset_index(drop = True)

In [ ]:
prob = (1 - confidence_level)

In [ ]:
VaR_Exact=-Bondwise_change_in_MTM['change_in_mtm'].quantile(prob)
VaR_Exact

In [ ]:
Bondwise_change_in_MTM2 = Bondwise_change_in_MTM.reset_index(drop=True)
Bondwise_change_in_MTM2

In [ ]:
###VaR nearby

In [ ]:
#Bondwise_change_in_MTM = sorted(Bondwise_change_in_MTM2, reverse = True)
nearby = np.floor(confidence_level * len(Bondwise_change_in_MTM2))
nearby = int(nearby)
nearby_nxt = nearby + 1

diff1 = pd.to_numeric(VaR_Exact + Bondwise_change_in_MTM2[nearby])
# diff2 = pd.to_numeric(- Bondwise_change_in_MTM2[nearby_nxt] - VaR_Exact)

# if (diff1 == diff2):
#     VaR_nearby_one_day = - Bondwise_change_in_MTM2[nearby + 1]
# elif (diff1 < diff2):
#     VaR_nearby_one_day = - Bondwise_change_in_MTM2[nearby]
# else:
#     VaR_nearby_one_day = - Bondwise_change_in_MTM2[nearby + 1]


In [ ]:
nearby_nxt

In [ ]:
nearby = math.floor(confidence_level * len(Bondwise_change_in_MTM))
nearby

In [ ]:
#nearby = np.floor(confidence_level * len(Bondwise_change_in_MTM))
nearby = int(nearby)
nearby_nxt = nearby + 1
nearby_nxt = int(nearby_nxt)

In [ ]:
nearby_nxt

In [ ]:
Bondwise_change_in_MTM = pd.DataFrame(Bondwise_change_in_MTM)

In [ ]:
Bondwise_change_in_MTM

In [ ]:
# diff2 = - Bondwise_change_in_MTM.iloc[nearby_nxt] - VaR_Exact

In [ ]:
# ##Corrected code 
# prob = (1 - confidence_level)
# decay_weights = []
# if (decay_factor==0):
#     VaR_exact_one_day = -(np.quantile(Bondwise_change_in_MTM, prob))
#     Portfolio_change_in_MTM = sorted(Bondwise_change_in_MTM, reverse = True)
#     nearby = np.floor(confidence_level * len(Bondwise_change_in_MTM))
#     nearby = int(nearby)
#     nearby_nxt = nearby + 1
    
#     diff1 = pd.to_numeric(VaR_exact_one_day + Bondwise_change_in_MTM[nearby])
#     diff2 = pd.to_numeric(- Bondwise_change_in_MTM[nearby_nxt] - VaR_exact_one_day)
                          
#     if (diff1 == diff2):
#         VaR_nearby_one_day = - Bondwise_change_in_MTM[nearby + 1]
#     elif (diff1 < diff2):
#         VaR_nearby_one_day = - Bondwise_change_in_MTM[nearby]
#     else:
#         VaR_nearby_one_day = - Bondwise_change_in_MTM[nearby + 1]



# else:
#     a = []
#     a = list((Result['env'].unique()))
#     a.pop()
#     for number in a:
#         decay_weights.append((1-decay_factor)*(decay_factor ** number))
    
#     total_mtm = Result.groupby('env')[['all_mtm']].sum
#     a= pd.DataFrame(range(0,(len(Result['env'].unique())-2)))
#     #hp['decay_factor'].pow(a)
#     #decay_weights = (1-decay_factor) * (hp['decay_factor'].pow(a))/((1 - decay_factor) ** (len(Result['env'].unique() -1)))
#     total_mtm = Result.groupby('env')['all_mtm'].sum()
#     total_mtm = list(total_mtm)
#     mtm_returns = []
    
#     for i in range(1,(len(Result['env'].unique() -1))):
#                    fv = total_mtm[0]
#                    returns = (total_mtm[i] - fv)/fv
#                    mtm_returns.append(returns)
#     #decay_weights.rename(columns = {0:'decay_weights'}, inplace = True)
#     decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
#     mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
#     mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
#     mtm = mtm.sort_values(['mtm_returns']).reset_index()
#     mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
#     alpha = 1-confidence_level
#     mtm1 = mtm.loc[mtm['cum_decay_weights'] < alpha]
    
# ######################################################################################################
    
#     if (len(mtm1['cum_decay_weights'])==0):
#                   VaR_exact_one_day = np.abs(- mtm['mtm_returns'][0] * total_mtm['mtm_bc'][0])



#                   VaR_exact_one_day = VaR_exact_one_day



#     else:
#                 index = len(mtm1['cum_decay_weights'])
                                         
#                 interpolated_return_exact = mtm['mtm_returns'][index-1] + ((mtm['mtm_returns'][index] - mtm['mtm_returns'][index-1]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) / (mtm['cum_decay_weights'][index] - mtm['cum_decay_weights'][index-1]))
    
#                 VaR_exact_one_day =  abs((total_mtm[0] * (interpolated_return_exact)))
    
#                 if((((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) < (mtm['cum_decay_weights'][index] - (1 - confidence_level) ))):
            
#                             interpolated_return_nearby = mtm['mtm_returns'][index-1]
#                             VAR_nearby_one_day = abs((total_mtm[0] * (interpolated_return_nearby))) 
                             
#                 else:
#                             index = len(mtm1['cum_decay_weights'])
#                             print(index)
#                             interpolated_return_nearby = mtm['mtm_returns'][index]
#                             print(interpolated_return_nearby)
#                             VAR_nearby_one_day = abs((total_mtm[0] * (interpolated_return_nearby)))
#                             print(VAR_nearby_one_day)

In [ ]:
# total_change_in_MTM2 = total_change_in_MTM.set_index('instrument')
# total_change_in_MTM2

# ##Segregating change_in_mtm according to the instrument
# Bondwise_change_in_MTM = (total_change_in_MTM2.assign(labels = total_change_in_MTM2.groupby(level = 0).cumcount())
#             .groupby([total_change_in_MTM2.index,'labels']).first()
#             .unstack('labels')
#             .sort_index(axis =1,level = 1)
#             .droplevel(1,axis = 1))
# Bondwise_change_in_MTM

# Bondwise_change_in_MTM = Bondwise_change_in_MTM.iloc[: , 1:]
# Bondwise_change_in_MTM

# #Bondwise_change_in_MTM.transpose()

# #Bondwise_change_in_MTM = Bondwise_change_in_MTM.sort_values("change_in_mtm",ascending=False, ignore_index=True)



# ##Sorting the dataframe in descending order
# trial_total = pd.DataFrame()
# for aa, gg in total_change_in_MTM1.groupby("instrument"):
#     gg = gg.iloc[: , 1:]   
#     gg = gg.reset_index(drop = True)
#     ga = gg.sort_values('change_in_mtm',ascending=False, ignore_index=True)
#     trial_total = trial_total.append(ga)

# trial_total

Bondwise var calculation fresh

In [ ]:
total_change_in_MTM

In [ ]:
type(total_change_in_MTM)

In [ ]:
# grouped = total_change_in_MTM_1.groupby("instrument")
# grouped

In [ ]:
# grouped = grouped.iloc[: , 1:]   


In [ ]:
trial_total = pd.DataFrame()
for aa, gg in total_change_in_MTM.groupby("instrument"):
    gg = gg.iloc[: , 1:]   
    #gg = gg.reset_index(drop = True)
    ga = gg.sort_values('change_in_mtm',ascending=False, ignore_index=True)
    trial_total = trial_total.append(ga)

trial_total